# 05 - Route Optimizer (VRP with Time Windows)

## Overview

This notebook demonstrates route optimization for dispatch candidates generated in Phase 4. We solve:

- **TSP** (Traveling Salesman Problem): Minimize total distance
- **VRPTW** (Vehicle Routing Problem with Time Windows): Respect delivery time constraints

## Approach

| Solver | Use Case | Complexity |
|--------|----------|------------|
| Nearest Neighbor | Baseline heuristic | O(n²) |
| OR-Tools TSP | Optimal for small instances | Exact/Metaheuristic |
| OR-Tools VRPTW | Time window constraints | Constraint Programming |

## Distance Calculation

We use **Haversine formula** for straight-line distance between coordinates. This is sufficient for ranking and estimation without requiring external routing APIs.

In [1]:
# Setup and Imports
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Standard imports
import json
from datetime import time

# Data handling
import pandas as pd

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
from IPython.display import display, HTML

# Project modules
from src.database import get_database_manager
from src.routing import (
    Location,
    RouteStop,
    RouteResult,
    DispatchWithRoute,
    RoutingConfig,
    load_routing_config,
    haversine_distance,
    build_distance_matrix,
    estimate_travel_time,
    solve_tsp_nearest_neighbor,
    solve_tsp_ortools,
    solve_vrptw_ortools,
    build_route_result,
    optimize_dispatch_route,
    optimize_all_dispatches,
    rank_dispatches_with_routes,
    create_route_map,
    create_multi_dispatch_map,
    save_route_map,
    export_dispatches_with_routes,
    init_route_cache,
    clear_route_cache,
    get_cache_stats,
)

print("Imports loaded successfully!")

Imports loaded successfully!


In [2]:
# Path configuration
DATA_DIR = project_root / "data"
CONFIG_DIR = DATA_DIR / "config"
DB_PATH = DATA_DIR / "processed" / "delivery.db"
OUTPUT_DIR = project_root / "output"
DISPATCH_DIR = OUTPUT_DIR / "dispatches"
MAPS_DIR = OUTPUT_DIR / "maps"

# Load configurations
ROUTING_CONFIG_PATH = CONFIG_DIR / "routing_config.json"
config = load_routing_config(ROUTING_CONFIG_PATH)

# Connect to database
db = get_database_manager(DB_PATH)

# Initialize persistent route cache (survives kernel restarts)
init_route_cache(db)

# Load dispatch candidates from Phase 4
CANDIDATES_PATH = DISPATCH_DIR / "dispatch_candidates.json"

print(f"Database: {DB_PATH}")
print(f"Routing Config: {ROUTING_CONFIG_PATH}")
print(f"Candidates: {CANDIDATES_PATH}")
print(f"\nDepot Location:")
print(f"  Name: {config.depot.name}")
print(f"  Coordinates: ({config.depot.latitude}, {config.depot.longitude})")
print(f"\nRouting Assumptions:")
print(f"  Average Speed: {config.average_speed_kmh} km/h")
print(f"  Service Time: {config.service_time_minutes} min/stop")
print(f"  Start Time: {config.default_start_time}")
print(f"  Preferred Solver: {config.preferred_solver}")

# Show cache status
cache_stats = get_cache_stats()
print(f"\nRoute Cache:")
print(f"  Cached routes: {cache_stats['size']}")
if cache_stats.get('by_solver'):
    for solver, count in cache_stats['by_solver'].items():
        print(f"    - {solver}: {count}")

Database: c:\Users\Santi\Desktop\CV\portafolio\Eco-Bags-Delivery-Optimizer\data\processed\delivery.db
Routing Config: c:\Users\Santi\Desktop\CV\portafolio\Eco-Bags-Delivery-Optimizer\data\config\routing_config.json
Candidates: c:\Users\Santi\Desktop\CV\portafolio\Eco-Bags-Delivery-Optimizer\output\dispatches\dispatch_candidates.json

Depot Location:
  Name: ECO-BAGS
  Coordinates: (-34.73231090267173, -58.295889556357935)

Routing Assumptions:
  Average Speed: 30 km/h
  Service Time: 15 min/stop
  Start Time: 08:00:00
  Preferred Solver: ortools

Route Cache:
  Cached routes: 28
    - ortools_tsp: 28


## 1. Load Dispatch Candidates

In [3]:
# Load dispatch candidates from JSON
with open(CANDIDATES_PATH, "r", encoding="utf-8") as f:
    candidates_data = json.load(f)

candidates = candidates_data["candidates"]
print(f"Loaded {len(candidates)} dispatch candidates")
print(f"\nCandidate Overview:")

for i, c in enumerate(candidates[:5], 1):
    summary = c.get("summary", {})
    print(f"  {i}. {c['candidate_id'][:30]}...")
    print(f"     Strategy: {c['strategy']}")
    print(f"     Orders: {summary.get('order_count', len(c.get('order_ids', [])))}")
    print(f"     Pallets: {summary.get('total_pallets', 'N/A')}")
    print(f"     Priority: {summary.get('total_priority', 'N/A')}")

Loaded 28 dispatch candidates

Candidate Overview:
  1. DISP-20260120-GREEDY-B86C...
     Strategy: greedy_efficiency
     Orders: 3
     Pallets: 8.41
     Priority: 2000083.92
  2. DISP-20260120-GREEDY-FD99...
     Strategy: greedy_mandatory_nearest
     Orders: 3
     Pallets: 8.21
     Priority: 2000039.03
  3. DISP-20260120-GREEDY-9D42...
     Strategy: greedy_priority
     Orders: 2
     Pallets: 7.5
     Priority: 1999998.0
  4. DISP-20260120-GREEDY-SUB-DC32...
     Strategy: greedy_efficiency
     Orders: 3
     Pallets: 8.31
     Priority: 2000065.27
  5. DISP-20260120-GREEDY-B27D...
     Strategy: greedy_zone_caba
     Orders: 3
     Pallets: 8.93
     Priority: 1000140.51


In [4]:
# Create summary DataFrame
candidates_df = pd.DataFrame([
    {
        "candidate_id": c["candidate_id"],
        "strategy": c["strategy"],
        "order_count": c.get("summary", {}).get("order_count", len(c.get("order_ids", []))),
        "total_pallets": c.get("summary", {}).get("total_pallets", 0),
        "total_priority": c.get("summary", {}).get("total_priority", 0),
        "zones": ", ".join(c.get("summary", {}).get("zones", [])),
        "is_single_zone": c.get("summary", {}).get("is_single_zone", False),
        "mandatory_count": c.get("summary", {}).get("mandatory_count", 0),
    }
    for c in candidates
])

print("Dispatch Candidates Summary:")
display(candidates_df)

Dispatch Candidates Summary:


,candidate_id,strategy,order_count,total_pallets,total_priority,zones,is_single_zone,mandatory_count
0,DISP-20260120-GREEDY-B86C,greedy_efficiency,3,8.41,2000083.92,"NORTH_ZONE, CABA",False,2
1,DISP-20260120-GREEDY-FD99,greedy_mandatory_nearest,3,8.21,2000039.03,"NORTH_ZONE, CABA",False,2
2,DISP-20260120-GREEDY-9D42,greedy_priority,2,7.50,1999998.00,"NORTH_ZONE, CABA",False,2
3,DISP-20260120-GREEDY-SUB-DC32,greedy_efficiency,3,8.31,2000065.27,"NORTH_ZONE, WEST_ZONE, CABA",False,2
4,DISP-20260120-GREEDY-B27D,greedy_zone_caba,3,8.93,1000140.51,CABA,True,1
5,DISP-20260120-GREEDY-SUB-93BD,greedy_zone_caba,4,8.00,1000167.70,CABA,True,1
6,DISP-20260120-GREEDY-SUB-0946,greedy_zone_north,3,7.36,1000118.55,NORTH_ZONE,True,1
7,DISP-20260120-GREEDY-SUB-35E3,greedy_zone_caba,3,7.09,1000081.78,CABA,True,1
8,DISP-20260120-GREEDY-04D1,greedy_zone_north,3,6.74,1000182.97,NORTH_ZONE,True,1
9,DISP-20260120-GREEDY-SUB-ACEE,greedy_zone_caba,2,8.91,149.48,CABA,True,0


## 2. Single Dispatch Demo

Let's pick one dispatch candidate and walk through the route optimization process step-by-step.

In [5]:
# Pick the first candidate for demo
demo_candidate = candidates[19]

print(f"Demo Candidate: {demo_candidate['candidate_id']}")
print(f"Strategy: {demo_candidate['strategy']}")

# Extract order IDs
order_ids = demo_candidate.get("order_ids", [])
if not order_ids and "orders" in demo_candidate:
    order_ids = [o["order_id"] for o in demo_candidate["orders"]]

print(f"Orders: {len(order_ids)}")
for oid in order_ids:
    print(f"  - {oid}")

Demo Candidate: DISP-20260120-DP_OPT-SUB-6138
Strategy: dp_optimal
Orders: 7
  - ORD-CE98BB79
  - ORD-C4A1485D
  - ORD-67B478B7
  - ORD-BAA10376
  - ORD-C8429AF5
  - ORD-85EB94F2
  - ORD-071F2BA3


In [6]:
# Get order coordinates from database
coords = db.get_order_coordinates(order_ids)

print(f"Retrieved coordinates for {len(coords)} orders:")
for oid, (lat, lon) in coords.items():
    print(f"  {oid}: ({lat:.4f}, {lon:.4f})")

# Build locations list (depot first)
locations = [config.depot]

orders_data = demo_candidate.get("orders", [])
order_map = {o["order_id"]: o for o in orders_data}

for order_id in order_ids:
    if order_id in coords:
        lat, lon = coords[order_id]
        order_info = order_map.get(order_id, {})
        loc = Location(
            id=order_id,
            name=order_info.get("client_name", order_id),
            latitude=lat,
            longitude=lon,
        )
        locations.append(loc)

print(f"\nTotal locations (including depot): {len(locations)}")

Retrieved coordinates for 7 orders:
  ORD-071F2BA3: (-34.5705, -58.4950)
  ORD-67B478B7: (-34.7190, -58.2572)
  ORD-85EB94F2: (-34.5893, -58.3944)
  ORD-BAA10376: (-34.6818, -58.3456)
  ORD-C4A1485D: (-34.3820, -58.7371)
  ORD-C8429AF5: (-34.6458, -58.5937)
  ORD-CE98BB79: (-34.7626, -58.1988)

Total locations (including depot): 8


In [7]:
# Show locations on map (before routing)
center_lat = sum(loc.latitude for loc in locations) / len(locations)
center_lon = sum(loc.longitude for loc in locations) / len(locations)

pre_route_map = folium.Map(location=[center_lat, center_lon]
                           , zoom_start=11,
                           tiles="cartodbpositron",)

# Add depot
folium.Marker(
    location=[config.depot.latitude, config.depot.longitude],
    popup="<b>Eco-Bags Factory</b><br>Depot Location",
    tooltip="Factory Depot",
    icon=folium.Icon(color="black", icon="industry", prefix="fa")
).add_to(pre_route_map)

# Add delivery locations
for i, loc in enumerate(locations[1:], 1):
    folium.Marker(
        location=[loc.latitude, loc.longitude],
        popup=f"<b>{loc.name}</b><br>Order: {loc.id}",
        icon=folium.Icon(color="blue", icon="box", prefix="fa"),
    ).add_to(pre_route_map)

print("📍 Delivery Locations (before routing):")
pre_route_map

📍 Delivery Locations (before routing):


In [8]:
# Build distance matrix
distance_matrix = build_distance_matrix(locations)

# Create labels for visualization
labels = ["Depot"] + [f"Order {i}" for i in range(1, len(locations))]

# Display as heatmap
fig = px.imshow(
    distance_matrix,
    labels=dict(x="To", y="From", color="Distance (km)"),
    x=labels,
    y=labels,
    color_continuous_scale="Blues",
    title="Distance Matrix (km)",
)

# Add text annotations
for i in range(len(distance_matrix)):
    for j in range(len(distance_matrix[i])):
        fig.add_annotation(
            x=j, y=i,
            text=f"{distance_matrix[i][j]:.1f}",
            showarrow=False,
            font=dict(size=10, color="white" if distance_matrix[i][j] > 10 else "black"),
        )

fig.update_layout(height=400)
fig.show()

## 3. Nearest Neighbor Baseline

The nearest neighbor heuristic always visits the closest unvisited location next. It's simple but not optimal.

In [9]:
# Solve with nearest neighbor
nn_order = solve_tsp_nearest_neighbor(locations)

print("Nearest Neighbor Solution:")
print(f"Visit Order: {nn_order}")
print("\nRoute:")

total_distance_nn = 0
for i, idx in enumerate(nn_order):
    loc = locations[idx]
    if i > 0:
        prev_loc = locations[nn_order[i-1]]
        dist = haversine_distance(
            prev_loc.latitude, prev_loc.longitude,
            loc.latitude, loc.longitude
        )
        total_distance_nn += dist
        print(f"  {i}. {loc.name} (+{dist:.2f} km)")
    else:
        print(f"  {i}. {loc.name} (START)")

# Add return to depot
last_loc = locations[nn_order[-1]]
return_dist = haversine_distance(
    last_loc.latitude, last_loc.longitude,
    config.depot.latitude, config.depot.longitude
)
total_distance_nn += return_dist
print(f"  → Return to Depot (+{return_dist:.2f} km)")
print(f"\nTotal Distance: {total_distance_nn:.2f} km")

Nearest Neighbor Solution:
Visit Order: [0, 3, 1, 4, 6, 7, 5, 2]

Route:
  0. ECO-BAGS (START)
  1. Supermercado Don Pedro (+3.83 km)
  2. Juguetería Fran Cardozo (+7.21 km)
  3. Comercial Rivadavia (+16.14 km)
  4. Comercial El Puente (+11.22 km)
  5. Mayorista Santa Fe (+9.44 km)
  6. Supermercado Norte (+12.31 km)
  7. Mayorista El Gaucho (+32.13 km)
  → Return to Depot (+56.12 km)

Total Distance: 148.40 km


In [10]:
# Build route result for nearest neighbor

nn_route = build_route_result(
    dispatch_id=demo_candidate["candidate_id"],
    locations=locations,
    visit_order=nn_order,
    start_time=config.default_start_time,
    service_time_minutes=config.service_time_minutes,
    avg_speed_kmh=config.average_speed_kmh,
    solver_used="nearest_neighbor",
)

print("Nearest Neighbor Route Result:")
print(f"  Total Distance: {nn_route.total_distance_km} km")
print(f"  Total Duration: {nn_route.total_duration_minutes} min")
print(f"  Start Time: {nn_route.route_start_time}")
print(f"  End Time: {nn_route.route_end_time}")

Nearest Neighbor Route Result:
  Total Distance: 148.4 km
  Total Duration: 406 min
  Start Time: 08:00:00
  End Time: 14:46:00


In [11]:
# Wrap route result in DispatchWithRoute for visualization

nn_dispatch = DispatchWithRoute(
    candidate_id=demo_candidate["candidate_id"],
    strategy=demo_candidate["strategy"],
    order_ids=order_ids,
    order_count=len(order_ids),
    total_pallets=demo_candidate.get("summary", {}).get("total_pallets", 0),
    total_priority=demo_candidate.get("summary", {}).get("total_priority", 0),
    zones=demo_candidate.get("summary", {}).get("zones", []),
    is_single_zone=demo_candidate.get("summary", {}).get("is_single_zone", False),
    has_mandatory=demo_candidate.get("summary", {}).get("mandatory_count", 0) > 0,
    mandatory_count=demo_candidate.get("summary", {}).get("mandatory_count", 0),
    route=nn_route,
    total_distance_km=nn_route.total_distance_km,
    total_duration_minutes=nn_route.total_duration_minutes,
)

# Create route map for nearest neighbor

nn_map = create_route_map(nn_dispatch, config.depot, config)
print("📍 Nearest Neighbor Route:")
nn_map

📍 Nearest Neighbor Route:


## 4. OR-Tools Optimization

OR-Tools uses metaheuristics (like Guided Local Search) to find better solutions than simple heuristics.

In [12]:
# Solve with OR-Tools
ortools_order = solve_tsp_ortools(locations, time_limit_seconds=30)

print("OR-Tools Solution:")
print(f"Visit Order: {ortools_order}")
print("\nRoute:")

total_distance_ortools = 0
for i, idx in enumerate(ortools_order):
    loc = locations[idx]
    if i > 0:
        prev_loc = locations[ortools_order[i-1]]
        dist = haversine_distance(
            prev_loc.latitude, prev_loc.longitude,
            loc.latitude, loc.longitude
        )
        total_distance_ortools += dist
        print(f"  {i}. {loc.name} (+{dist:.2f} km)")
    else:
        print(f"  {i}. {loc.name} (START)")

# Add return to depot
last_loc = locations[ortools_order[-1]]
return_dist = haversine_distance(
    last_loc.latitude, last_loc.longitude,
    config.depot.latitude, config.depot.longitude
)
total_distance_ortools += return_dist
print(f"  → Return to Depot (+{return_dist:.2f} km)")
print(f"\nTotal Distance: {total_distance_ortools:.2f} km")

OR-Tools Solution:
Visit Order: [0, 1, 3, 4, 6, 7, 2, 5]

Route:
  0. ECO-BAGS (START)
  1. Juguetería Fran Cardozo (+9.49 km)
  2. Supermercado Don Pedro (+7.21 km)
  3. Comercial Rivadavia (+9.08 km)
  4. Comercial El Puente (+11.22 km)
  5. Mayorista Santa Fe (+9.44 km)
  6. Mayorista El Gaucho (+30.52 km)
  7. Supermercado Norte (+32.13 km)
  → Return to Depot (+28.88 km)

Total Distance: 137.96 km


In [13]:
# Build route result for OR-Tools
ortools_route = build_route_result(
    dispatch_id=demo_candidate["candidate_id"],
    locations=locations,
    visit_order=ortools_order,
    start_time=config.default_start_time,
    service_time_minutes=config.service_time_minutes,
    avg_speed_kmh=config.average_speed_kmh,
    solver_used="ortools_tsp",
)

print("OR-Tools Route Result:")
print(f"  Total Distance: {ortools_route.total_distance_km} km")
print(f"  Total Duration: {ortools_route.total_duration_minutes} min")
print(f"  Start Time: {ortools_route.route_start_time}")
print(f"  End Time: {ortools_route.route_end_time}")

OR-Tools Route Result:
  Total Distance: 137.96 km
  Total Duration: 385 min
  Start Time: 08:00:00
  End Time: 14:25:00


In [14]:
# Compare solvers
improvement = (total_distance_nn - total_distance_ortools) / total_distance_nn * 100

print("\n" + "=" * 50)
print("SOLVER COMPARISON")
print("=" * 50)
print(f"\nNearest Neighbor: {total_distance_nn:.2f} km")
print(f"OR-Tools:         {total_distance_ortools:.2f} km")
print(f"\nImprovement: {improvement:.1f}%")

if improvement > 0:
    print(f"OR-Tools saved {total_distance_nn - total_distance_ortools:.2f} km")
else:
    print("Nearest Neighbor found optimal solution for this small instance")


SOLVER COMPARISON

Nearest Neighbor: 148.40 km
OR-Tools:         137.96 km

Improvement: 7.0%
OR-Tools saved 10.44 km


In [15]:
# Create OR-Tools route map
# Wrap route result in DispatchWithRoute for visualization
ortools_dispatch = DispatchWithRoute(
    candidate_id=demo_candidate["candidate_id"],
    strategy=demo_candidate["strategy"],
    order_ids=order_ids,
    order_count=len(order_ids),
    total_pallets=demo_candidate.get("summary", {}).get("total_pallets", 0),
    total_priority=demo_candidate.get("summary", {}).get("total_priority", 0),
    zones=demo_candidate.get("summary", {}).get("zones", []),
    is_single_zone=demo_candidate.get("summary", {}).get("is_single_zone", False),
    has_mandatory=demo_candidate.get("summary", {}).get("mandatory_count", 0) > 0,
    mandatory_count=demo_candidate.get("summary", {}).get("mandatory_count", 0),
    route=ortools_route,
    total_distance_km=ortools_route.total_distance_km,
    total_duration_minutes=ortools_route.total_duration_minutes,
)

ortools_map = create_route_map(ortools_dispatch, config.depot, config)
print("📍 OR-Tools Optimized Route:")
ortools_map

📍 OR-Tools Optimized Route:


## 5. Time Windows Handling (VRPTW)

When orders have delivery time windows, we use VRPTW (Vehicle Routing Problem with Time Windows).

In [16]:
# Check if any orders have time windows
has_time_windows = any(
    loc.time_window_start is not None for loc in locations[1:]
)

print(f"Orders with time windows: {has_time_windows}")

if has_time_windows:
    print("\nTime Windows:")
    for loc in locations[1:]:
        if loc.time_window_start:
            print(f"  {loc.name}: {loc.time_window_start} - {loc.time_window_end}")
else:
    print("\nNo time windows specified - using regular TSP.")
    print("Creating sample time windows for demonstration...")
    
    # Add sample time windows for demo
    demo_locations = [config.depot]
    for i, loc in enumerate(locations[1:]):
        # Alternate between morning and afternoon windows
        if i % 2 == 0:
            tw_start = time(9, 0)
            tw_end = time(12, 0)
        else:
            tw_start = time(14, 0)
            tw_end = time(17, 0)
        
        demo_loc = Location(
            id=loc.id,
            name=loc.name,
            latitude=loc.latitude,
            longitude=loc.longitude,
            time_window_start=tw_start,
            time_window_end=tw_end,
        )
        demo_locations.append(demo_loc)
        print(f"  {demo_loc.name}: {tw_start} - {tw_end}")

Orders with time windows: False

No time windows specified - using regular TSP.
Creating sample time windows for demonstration...
  Juguetería Fran Cardozo: 09:00:00 - 12:00:00
  Mayorista El Gaucho: 14:00:00 - 17:00:00
  Supermercado Don Pedro: 09:00:00 - 12:00:00
  Comercial Rivadavia: 14:00:00 - 17:00:00
  Supermercado Norte: 09:00:00 - 12:00:00
  Comercial El Puente: 14:00:00 - 17:00:00
  Mayorista Santa Fe: 09:00:00 - 12:00:00


In [17]:
# Solve VRPTW
if has_time_windows:
    vrptw_locations = locations
else:
    vrptw_locations = demo_locations

vrptw_order, feasible = solve_vrptw_ortools(
    vrptw_locations,
    start_time=config.default_start_time,
    service_time_minutes=config.service_time_minutes,
    avg_speed_kmh=config.average_speed_kmh,
    time_limit_seconds=10,
)

print(f"\nVRPTW Solution:")
print(f"Feasible: {feasible}")
print(f"Visit Order: {vrptw_order}")

if feasible:
    vrptw_route = build_route_result(
        dispatch_id=demo_candidate["candidate_id"],
        locations=vrptw_locations,
        visit_order=vrptw_order,
        start_time=config.default_start_time,
        service_time_minutes=config.service_time_minutes,
        avg_speed_kmh=config.average_speed_kmh,
        solver_used="ortools_vrptw",
        feasible=feasible,
    )
    
    print(f"\nRoute with Time Windows:")
    print(f"  Total Distance: {vrptw_route.total_distance_km} km")
    print(f"  Total Duration: {vrptw_route.total_duration_minutes} min")
    
    print(f"\nSchedule:")
    for stop in vrptw_route.stops:
        tw = ""
        if stop.location.time_window_start:
            tw = f" [TW: {stop.location.time_window_start}-{stop.location.time_window_end}]"
        wait = f" (wait {stop.wait_time_minutes} min)" if stop.wait_time_minutes > 0 else ""
        print(f"  {stop.sequence}. {stop.location.name}: Arrive {stop.arrival_time}{tw}{wait}")
else:
    print("\n⚠️ Time windows cannot be satisfied!")
    print("Falling back to TSP solution without time constraints.")


VRPTW Solution:
Feasible: True
Visit Order: [0, 1, 3, 7, 5, 2, 6, 4]

Route with Time Windows:
  Total Distance: 145.84 km
  Total Duration: 521 min

Schedule:
  0. ECO-BAGS: Arrive 08:00:00
  1. Juguetería Fran Cardozo: Arrive 09:00:00 [TW: 09:00:00-12:00:00] (wait 41 min)
  2. Supermercado Don Pedro: Arrive 09:30:00 [TW: 09:00:00-12:00:00]
  3. Mayorista Santa Fe: Arrive 10:40:00 [TW: 09:00:00-12:00:00]
  4. Supermercado Norte: Arrive 11:20:00 [TW: 09:00:00-12:00:00]
  5. Mayorista El Gaucho: Arrive 14:00:00 [TW: 14:00:00-17:00:00] (wait 80 min)
  6. Comercial El Puente: Arrive 15:33:00 [TW: 14:00:00-17:00:00]
  7. Comercial Rivadavia: Arrive 16:11:00 [TW: 14:00:00-17:00:00]


## 6. Optimize All Dispatches

Now let's optimize routes for all dispatch candidates.

In [18]:
# Optimize all dispatch candidates (with caching)
print("Optimizing routes for all dispatch candidates...")
print("Routes are cached - subsequent runs with same orders will be faster.\n")

# Show initial cache state
cache_before = get_cache_stats()
print(f"Cache before: {cache_before['size']} entries")

all_dispatches = []
cache_hits = 0
for i, candidate in enumerate(candidates, 1):
    try:
        cache_size_before = get_cache_stats()["size"]
        result = optimize_dispatch_route(candidate, config, db)
        cache_size_after = get_cache_stats()["size"]
        
        # Check if this was a cache hit (cache size unchanged)
        from_cache = " (cached)" if cache_size_before == cache_size_after else ""
        if from_cache:
            cache_hits += 1
        
        all_dispatches.append(result)
        print(f"  {i}/{len(candidates)}: {candidate['strategy'][:20]:<20} "
              f"→ {result.total_distance_km:.1f} km, {result.total_duration_minutes} min{from_cache}")
    except Exception as e:
        print(f"  {i}/{len(candidates)}: {candidate['strategy'][:20]:<20} → ERROR: {e}")

# Show final cache state
cache_after = get_cache_stats()
print(f"\n✓ Optimized {len(all_dispatches)} dispatches")
print(f"  Cache hits: {cache_hits}")
print(f"  Cache size: {cache_after['size']} entries")

Optimizing routes for all dispatch candidates...
Routes are cached - subsequent runs with same orders will be faster.

Cache before: 28 entries
  1/28: greedy_efficiency    → 85.0 km, 217 min (cached)
  2/28: greedy_mandatory_nea → 85.2 km, 218 min (cached)
  3/28: greedy_priority      → 84.6 km, 201 min (cached)
  4/28: greedy_efficiency    → 95.1 km, 237 min (cached)
  5/28: greedy_zone_caba     → 50.9 km, 149 min (cached)
  6/28: greedy_zone_caba     → 55.3 km, 173 min (cached)
  7/28: greedy_zone_north    → 115.2 km, 277 min (cached)
  8/28: greedy_zone_caba     → 54.8 km, 157 min (cached)
  9/28: greedy_zone_north    → 114.1 km, 275 min (cached)
  10/28: greedy_zone_caba     → 36.6 km, 105 min (cached)
  11/28: greedy_zone_north    → 127.9 km, 303 min (cached)
  12/28: greedy_zone_south    → 40.0 km, 142 min (cached)
  13/28: greedy_best_fit      → 24.6 km, 65 min (cached)
  14/28: greedy_zone_west     → 95.9 km, 239 min (cached)
  15/28: greedy_zone_spillove → 40.5 km, 128 min (c

In [19]:
# Rank dispatches with routes
ranked_dispatches = rank_dispatches_with_routes(
    all_dispatches,
    priority_weight=0.5,
    distance_weight=0.4,
    utilization_weight=0.1,
)

print("Dispatches ranked by combined score:")
print("(Priority 50%, Distance Efficiency 40%, Utilization 10%)\n")
for i, d in enumerate(ranked_dispatches[:5], 1):
    print(f"  #{i}: {d.strategy}")
    print(f"      Orders: {d.order_count} | Pallets: {d.total_pallets}")
    print(f"      Priority: {d.total_priority:.0f} | Distance: {d.total_distance_km:.1f} km")

Dispatches ranked by combined score:
(Priority 50%, Distance Efficiency 40%, Utilization 10%)

  #1: greedy_zone_caba
      Orders: 3 | Pallets: 8.93
      Priority: 1000141 | Distance: 50.9 km
  #2: greedy_zone_caba
      Orders: 4 | Pallets: 8.0
      Priority: 1000168 | Distance: 55.3 km
  #3: greedy_zone_caba
      Orders: 3 | Pallets: 7.09
      Priority: 1000082 | Distance: 54.8 km
  #4: greedy_efficiency
      Orders: 3 | Pallets: 8.41
      Priority: 2000084 | Distance: 85.0 km
  #5: greedy_mandatory_nearest
      Orders: 3 | Pallets: 8.21
      Priority: 2000039 | Distance: 85.2 km


## 7. Results Table

In [20]:
# Create comprehensive results DataFrame
results_df = pd.DataFrame([
    {
        "rank": i + 1,
        "candidate_id": d.candidate_id,
        "strategy": d.strategy,
        "order_count": d.order_count,
        "total_priority": d.total_priority,
        "zones": ", ".join(d.zones) if d.zones else "N/A",
        "is_single_zone": d.is_single_zone,
        "has_mandatory": d.has_mandatory,
        "mandatory_count": d.mandatory_count,
        "total_pallets": d.total_pallets,
        "total_distance_km": d.total_distance_km,
        "total_duration_min": d.total_duration_minutes,
        "solver": d.route.solver_used,
        "feasible": d.route.feasible,
    }
    for i, d in enumerate(ranked_dispatches)
])

print("Complete Results Table:")
display(results_df)

Complete Results Table:


,rank,candidate_id,strategy,order_count,total_priority,zones,is_single_zone,has_mandatory,mandatory_count,total_pallets,total_distance_km,total_duration_min,solver,feasible
0,1,DISP-20260120-GREEDY-B27D,greedy_zone_caba,3,1000140.51,CABA,True,True,1,8.93,50.93,149,ortools_tsp,True
1,2,DISP-20260120-GREEDY-SUB-93BD,greedy_zone_caba,4,1000167.70,CABA,True,True,1,8.00,55.33,173,ortools_tsp,True
2,3,DISP-20260120-GREEDY-SUB-35E3,greedy_zone_caba,3,1000081.78,CABA,True,True,1,7.09,54.83,157,ortools_tsp,True
3,4,DISP-20260120-GREEDY-B86C,greedy_efficiency,3,2000083.92,"NORTH_ZONE, CABA",False,True,2,8.41,84.97,217,ortools_tsp,True
4,5,DISP-20260120-GREEDY-FD99,greedy_mandatory_nearest,3,2000039.03,"NORTH_ZONE, CABA",False,True,2,8.21,85.20,218,ortools_tsp,True
5,6,DISP-20260120-GREEDY-9D42,greedy_priority,2,1999998.00,"NORTH_ZONE, CABA",False,True,2,7.50,84.64,201,ortools_tsp,True
6,7,DISP-20260120-GREEDY-SUB-DC32,greedy_efficiency,3,2000065.27,"NORTH_ZONE, WEST_ZONE, CABA",False,True,2,8.31,95.09,237,ortools_tsp,True
7,8,DISP-20260120-GREEDY-SUB-0946,greedy_zone_north,3,1000118.55,NORTH_ZONE,True,True,1,7.36,115.21,277,ortools_tsp,True
8,9,DISP-20260120-GREEDY-04D1,greedy_zone_north,3,1000182.97,NORTH_ZONE,True,True,1,6.74,114.07,275,ortools_tsp,True
9,10,DISP-20260120-GREEDY-2DF4,greedy_zone_south,4,212.07,SOUTH_ZONE,True,False,0,8.06,39.95,142,ortools_tsp,True


In [21]:
# Interactive table with Plotly
fig = go.Figure(data=[go.Table(
    header=dict(
        values=["Rank", "Strategy", "Orders", "Priority", "Zones", "Mandatory", 
                "Pallets", "Distance (km)", "Duration (min)"],
        fill_color="#2c3e50",
        font=dict(color="white", size=12),
        align="left",
    ),
    cells=dict(
        values=[
            results_df["rank"],
            results_df["strategy"],
            results_df["order_count"],
            results_df["total_priority"].apply(lambda x: f"{x:,.0f}"),
            results_df["zones"],
            results_df["mandatory_count"],
            results_df["total_pallets"],
            results_df["total_distance_km"].apply(lambda x: f"{x:.1f}"),
            results_df["total_duration_min"],
        ],
        fill_color=["#ecf0f1" if i % 2 == 0 else "white" for i in range(len(results_df))],
        align="left",
    ),
)])

fig.update_layout(
    title="Dispatch Candidates with Route Optimization Results",
    height=400,
)
fig.show()

## 8. Visualizations

In [22]:
# Bar chart: Distance and Priority side by side
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Route Distance by Strategy", "Total Priority by Strategy"),
    horizontal_spacing=0.12
)

df_sorted = results_df.sort_values("total_distance_km")
colors = ["#2E8B57" if sz else "#FF6B6B" for sz in df_sorted["is_single_zone"]]

fig.add_trace(
    go.Bar(x=df_sorted["strategy"], y=df_sorted["total_distance_km"], marker_color=colors, name="Distance"),
    row=1, col=1
)

df_sorted_prio = results_df.sort_values("total_priority", ascending=False)
colors_prio = ["#2E8B57" if sz else "#FF6B6B" for sz in df_sorted_prio["is_single_zone"]]

fig.add_trace(
    go.Bar(x=df_sorted_prio["strategy"], y=df_sorted_prio["total_priority"], marker_color=colors_prio, name="Priority"),
    row=1, col=2
)

fig.update_layout(
    height=450, 
    showlegend=False,
    title_text="Dispatch Comparison: Distance vs Priority",
)
fig.update_xaxes(tickangle=45)
fig.update_yaxes(title_text="Distance (km)", row=1, col=1)
fig.update_yaxes(title_text="Priority Score", row=1, col=2)
fig.show()

In [23]:
# Scatter: Priority vs Distance with annotations
fig = px.scatter(
    results_df,
    x="total_distance_km",
    y="total_priority",
    size="order_count",
    color="is_single_zone",
    color_discrete_map={True: "#2E8B57", False: "#FF6B6B"},
    hover_data=["strategy", "zones", "total_pallets"],
    title="Priority vs Distance Trade-off Analysis",
    labels={
        "total_distance_km": "Distance (km)",
        "total_priority": "Priority Score",
        "is_single_zone": "Single Zone",
    },
)

# Mark top dispatch
top = results_df.iloc[0]
fig.add_annotation(
    x=top["total_distance_km"], y=top["total_priority"],
    text="Top Ranked", showarrow=True, arrowhead=2,
    ax=40, ay=-40, font=dict(color="#2E8B57", size=12)
)

# Add quadrant shading annotation
fig.add_annotation(
    x=results_df["total_distance_km"].min() + 5,
    y=results_df["total_priority"].max() - 1000,
    text="✓ Ideal Zone",
    showarrow=False,
    font=dict(color="#2E8B57", size=14),
    opacity=0.7
)

fig.update_layout(height=500)
fig.show()

In [24]:
# Top 5 Dispatches Ranked - Horizontal bar chart
top5 = results_df.head(5).copy()
top5["rank_label"] = [f"#{i+1} {s}" for i, s in enumerate(top5["strategy"])]
top5 = top5.iloc[::-1]  # Reverse for proper ordering

fig = go.Figure()

# Priority bars
fig.add_trace(go.Bar(
    y=top5["rank_label"],
    x=top5["total_priority"],
    orientation="h",
    name="Priority Score",
    marker_color="#2E8B57",
    text=top5["total_priority"].apply(lambda x: f"{x:,.0f}"),
    textposition="inside",
))

fig.update_layout(
    title="Top 5 Dispatch Candidates by Priority",
    xaxis_title="Priority Score",
    yaxis_title="",
    height=350,
    showlegend=False,
    margin=dict(l=200),
)
fig.show()

In [25]:
# Duration vs Distance bubble chart with zone coloring
fig = px.scatter(
    results_df,
    x="total_distance_km",
    y="total_duration_min",
    size="total_pallets",
    color="zones",
    hover_data=["strategy", "total_priority", "order_count"],
    title="Route Duration vs Distance by Zone",
    labels={
        "total_distance_km": "Distance (km)",
        "total_duration_min": "Duration (min)",
        "zones": "Zone(s)",
    },
    color_discrete_sequence=["#2E8B57", "#FF6B6B", "#4ECDC4", "#9B59B6", "#F39C12"]
)

fig.update_layout(height=450)
fig.show()

## 8.1 Visualizations Without Mandatory Orders

The mandatory orders have an artificially high priority score (2,000,000) to ensure they are always included in dispatches. This skews the visualizations and makes it harder to compare the real priority differences between strategies. Below, we filter out dispatches that include mandatory orders to see a cleaner comparison.

In [26]:
# Filter results to exclude dispatches with mandatory orders
results_no_mandatory = results_df[results_df["mandatory_count"] == 0].copy()
results_no_mandatory = results_no_mandatory.reset_index(drop=True)
results_no_mandatory["rank"] = range(1, len(results_no_mandatory) + 1)

print(f"Total dispatches: {len(results_df)}")
print(f"Dispatches without mandatory orders: {len(results_no_mandatory)}")
print(f"Dispatches with mandatory orders (excluded): {len(results_df) - len(results_no_mandatory)}")

Total dispatches: 28
Dispatches without mandatory orders: 19
Dispatches with mandatory orders (excluded): 9


In [27]:
# Results table without mandatory orders
print("Results Table (Excluding Mandatory Orders):")
display(results_no_mandatory)

Results Table (Excluding Mandatory Orders):


,rank,candidate_id,strategy,order_count,total_priority,zones,is_single_zone,has_mandatory,mandatory_count,total_pallets,total_distance_km,total_duration_min,solver,feasible
0,1,DISP-20260120-GREEDY-2DF4,greedy_zone_south,4,212.07,SOUTH_ZONE,True,False,0,8.06,39.95,142,ortools_tsp,True
1,2,DISP-20260120-GREEDY-SUB-B422,greedy_best_fit,1,63.56,CABA,True,False,0,8.00,24.61,65,ortools_tsp,True
2,3,DISP-20260120-GREEDY-SUB-ACEE,greedy_zone_caba,2,149.48,CABA,True,False,0,8.91,36.63,105,ortools_tsp,True
3,4,DISP-20260120-GREEDY-SUB-6F62,greedy_zone_spillover,3,183.26,CABA,True,False,0,7.89,40.46,128,ortools_tsp,True
4,5,DISP-20260120-GREEDY-SUB-9A84,greedy_zone_south,3,192.49,SOUTH_ZONE,True,False,0,6.91,39.81,126,ortools_tsp,True
5,6,DISP-20260120-GREEDY-SUB-D4F8,greedy_zone_south,4,204.50,SOUTH_ZONE,True,False,0,7.28,54.33,171,ortools_tsp,True
6,7,DISP-20260120-GREEDY-SUB-0881,greedy_zone_west,2,159.59,WEST_ZONE,True,False,0,5.62,62.90,157,ortools_tsp,True
7,8,DISP-20260120-GREEDY-56E6,greedy_zone_west,3,225.94,WEST_ZONE,True,False,0,7.94,95.92,239,ortools_tsp,True
8,9,DISP-20260120-GREEDY-SUB-F44C,greedy_zone_west,2,158.67,WEST_ZONE,True,False,0,7.13,95.10,223,ortools_tsp,True
9,10,DISP-20260120-GREEDY-SUB-5E4E,greedy_priority,3,294.10,"NORTH_ZONE, SOUTH_ZONE, WEST_ZONE",False,False,0,7.82,113.82,274,ortools_tsp,True


In [28]:
# Interactive table with Plotly (no mandatory orders)
fig = go.Figure(data=[go.Table(
    header=dict(
        values=["Rank", "Strategy", "Orders", "Priority", "Zones", 
                "Pallets", "Distance (km)", "Duration (min)"],
        fill_color="#2c3e50",
        font=dict(color="white", size=12),
        align="left",
    ),
    cells=dict(
        values=[
            results_no_mandatory["rank"],
            results_no_mandatory["strategy"],
            results_no_mandatory["order_count"],
            results_no_mandatory["total_priority"].apply(lambda x: f"{x:,.0f}"),
            results_no_mandatory["zones"],
            results_no_mandatory["total_pallets"],
            results_no_mandatory["total_distance_km"].apply(lambda x: f"{x:.1f}"),
            results_no_mandatory["total_duration_min"],
        ],
        fill_color=[["#ecf0f1" if i % 2 == 0 else "white" for i in range(len(results_no_mandatory))]],
        align="left",
    ),
)])

fig.update_layout(
    title="Dispatch Candidates (Excluding Mandatory Orders)",
    height=400,
)
fig.show()

In [29]:
# Bar chart: Distance and Priority side by side (no mandatory)
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Route Distance by Strategy", "Total Priority by Strategy"),
    horizontal_spacing=0.12
)

df_sorted = results_no_mandatory.sort_values("total_distance_km")
colors = ["#2E8B57" if sz else "#FF6B6B" for sz in df_sorted["is_single_zone"]]

fig.add_trace(
    go.Bar(x=df_sorted["strategy"], y=df_sorted["total_distance_km"], marker_color=colors, name="Distance"),
    row=1, col=1
)

df_sorted_prio = results_no_mandatory.sort_values("total_priority", ascending=False)
colors_prio = ["#2E8B57" if sz else "#FF6B6B" for sz in df_sorted_prio["is_single_zone"]]

fig.add_trace(
    go.Bar(x=df_sorted_prio["strategy"], y=df_sorted_prio["total_priority"], marker_color=colors_prio, name="Priority"),
    row=1, col=2
)

fig.update_layout(
    height=450, 
    showlegend=False,
    title_text="Dispatch Comparison (Excluding Mandatory): Distance vs Priority",
)
fig.update_xaxes(tickangle=45)
fig.update_yaxes(title_text="Distance (km)", row=1, col=1)
fig.update_yaxes(title_text="Priority Score", row=1, col=2)
fig.show()

In [30]:
# Scatter: Priority vs Distance with annotations (no mandatory)
fig = px.scatter(
    results_no_mandatory,
    x="total_distance_km",
    y="total_priority",
    size="order_count",
    color="is_single_zone",
    color_discrete_map={True: "#2E8B57", False: "#FF6B6B"},
    hover_data=["strategy", "zones", "total_pallets"],
    title="Priority vs Distance Trade-off (Excluding Mandatory Orders)",
    labels={
        "total_distance_km": "Distance (km)",
        "total_priority": "Priority Score",
        "is_single_zone": "Single Zone",
    },
)

# Mark top dispatch in this filtered set
if len(results_no_mandatory) > 0:
    top = results_no_mandatory.iloc[0]
    fig.add_annotation(
        x=top["total_distance_km"], y=top["total_priority"],
        text="Top Ranked", showarrow=True, arrowhead=2,
        ax=40, ay=-40, font=dict(color="#2E8B57", size=12)
    )
    
    # Add quadrant shading annotation
    fig.add_annotation(
        x=results_no_mandatory["total_distance_km"].min() + 5,
        y=results_no_mandatory["total_priority"].max() * 0.95,
        text="✓ Ideal Zone",
        showarrow=False,
        font=dict(color="#2E8B57", size=14),
        opacity=0.7
    )

fig.update_layout(height=500)
fig.show()

In [31]:
# Top 5 Dispatches Ranked - Horizontal bar chart (no mandatory)
if len(results_no_mandatory) >= 1:
    top5_nm = results_no_mandatory.head(5).copy()
    top5_nm["rank_label"] = [f"#{i+1} {s}" for i, s in enumerate(top5_nm["strategy"])]
    top5_nm = top5_nm.iloc[::-1]  # Reverse for proper ordering

    fig = go.Figure()

    # Priority bars
    fig.add_trace(go.Bar(
        y=top5_nm["rank_label"],
        x=top5_nm["total_priority"],
        orientation="h",
        name="Priority Score",
        marker_color="#2E8B57",
        text=top5_nm["total_priority"].apply(lambda x: f"{x:,.0f}"),
        textposition="inside",
    ))

    fig.update_layout(
        title="Top 5 Dispatch Candidates by Priority (Excluding Mandatory)",
        xaxis_title="Priority Score",
        yaxis_title="",
        height=350,
        showlegend=False,
        margin=dict(l=200),
    )
    fig.show()
else:
    print("Not enough dispatches without mandatory orders for top 5 chart.")

## 9. Route Maps

In [32]:
# Top-ranked dispatch route map
top_dispatch = ranked_dispatches[0]
print(f"Top Ranked Dispatch: {top_dispatch.strategy}")
print(f"  Orders: {top_dispatch.order_count}")
print(f"  Distance: {top_dispatch.total_distance_km} km")
print(f"  Duration: {top_dispatch.total_duration_minutes} min")
print(f"  Priority: {top_dispatch.total_priority}")

top_map = create_route_map(top_dispatch, config.depot, config)
print("\n📍 Top Ranked Dispatch Route:")
top_map

Top Ranked Dispatch: greedy_zone_caba
  Orders: 3
  Distance: 50.93 km
  Duration: 149 min
  Priority: 1000140.51

📍 Top Ranked Dispatch Route:


In [33]:
# Best single-zone dispatch
single_zone_dispatches = [d for d in ranked_dispatches if d.is_single_zone]

if single_zone_dispatches:
    best_single = single_zone_dispatches[0]
    print(f"Best Single-Zone Dispatch: {best_single.strategy}")
    print(f"  Zone: {best_single.zones}")
    print(f"  Orders: {best_single.order_count}")
    print(f"  Distance: {best_single.total_distance_km} km")
    print(f"  Priority: {best_single.total_priority}")
    
    single_map = create_route_map(best_single, config.depot, config)
    print("\n📍 Best Single-Zone Dispatch Route:")
    display(single_map)
else:
    print("No single-zone dispatches available.")

Best Single-Zone Dispatch: greedy_zone_caba
  Zone: ['CABA']
  Orders: 3
  Distance: 50.93 km
  Priority: 1000140.51

📍 Best Single-Zone Dispatch Route:


In [34]:
# Comparison map: Top 3 routes overlaid
multi_map = create_multi_dispatch_map(
    ranked_dispatches,
    config.depot,
    config,
    show_top_n=3,
)

print("📍 Top 3 Routes Comparison (see legend on map):")
for i in range(min(3, len(ranked_dispatches))):
    d = ranked_dispatches[i]
    print(f"  Route #{i+1}: {d.strategy} ({d.order_count} orders, {d.total_distance_km:.1f} km)")
multi_map

📍 Top 3 Routes Comparison (see legend on map):
  Route #1: greedy_zone_caba (3 orders, 50.9 km)
  Route #2: greedy_zone_caba (4 orders, 55.3 km)
  Route #3: greedy_zone_caba (3 orders, 54.8 km)


In [35]:
# Save maps to output folder
MAPS_DIR.mkdir(parents=True, exist_ok=True)

# Save top dispatch route
save_route_map(top_map, MAPS_DIR / "top_dispatch_route.html")
print(f"✓ Saved: {MAPS_DIR / 'top_dispatch_route.html'}")

# Save best single-zone route
if single_zone_dispatches:
    save_route_map(single_map, MAPS_DIR / "best_single_zone_route.html")
    print(f"✓ Saved: {MAPS_DIR / 'best_single_zone_route.html'}")

# Save comparison map
save_route_map(multi_map, MAPS_DIR / "top3_routes_comparison.html")
print(f"✓ Saved: {MAPS_DIR / 'top3_routes_comparison.html'}")

✓ Saved: c:\Users\Santi\Desktop\CV\portafolio\Eco-Bags-Delivery-Optimizer\output\maps\top_dispatch_route.html
✓ Saved: c:\Users\Santi\Desktop\CV\portafolio\Eco-Bags-Delivery-Optimizer\output\maps\best_single_zone_route.html
✓ Saved: c:\Users\Santi\Desktop\CV\portafolio\Eco-Bags-Delivery-Optimizer\output\maps\top3_routes_comparison.html


## 10. Final Recommendations and Export

In [36]:
# Top 3 recommendations
print("=" * 70)
print("TOP 3 DISPATCH RECOMMENDATIONS")
print("=" * 70)

for i, d in enumerate(ranked_dispatches[:3], 1):
    print(f"\n{'─' * 70}")
    print(f"RANK #{i}: {d.candidate_id}")
    print(f"{'─' * 70}")
    print(f"Strategy:       {d.strategy}")
    print(f"Orders:         {d.order_count}")
    print(f"Total Pallets:  {d.total_pallets}")
    print(f"Total Priority: {d.total_priority:,.0f}")
    print(f"Zones:          {', '.join(d.zones) if d.zones else 'N/A'}")
    print(f"Mandatory:      {d.mandatory_count} orders")
    print(f"\nRoute Details:")
    print(f"  Distance:     {d.total_distance_km:.1f} km")
    print(f"  Duration:     {d.total_duration_minutes} min")
    print(f"  Start Time:   {d.route.route_start_time}")
    print(f"  End Time:     {d.route.route_end_time}")
    print(f"  Solver:       {d.route.solver_used}")
    print(f"  Feasible:     {d.route.feasible}")

TOP 3 DISPATCH RECOMMENDATIONS

──────────────────────────────────────────────────────────────────────
RANK #1: DISP-20260120-GREEDY-B27D
──────────────────────────────────────────────────────────────────────
Strategy:       greedy_zone_caba
Orders:         3
Total Pallets:  8.93
Total Priority: 1,000,141
Zones:          CABA
Mandatory:      1 orders

Route Details:
  Distance:     50.9 km
  Duration:     149 min
  Start Time:   08:00:00
  End Time:     10:29:00
  Solver:       ortools_tsp
  Feasible:     True

──────────────────────────────────────────────────────────────────────
RANK #2: DISP-20260120-GREEDY-SUB-93BD
──────────────────────────────────────────────────────────────────────
Strategy:       greedy_zone_caba
Orders:         4
Total Pallets:  8.0
Total Priority: 1,000,168
Zones:          CABA
Mandatory:      1 orders

Route Details:
  Distance:     55.3 km
  Duration:     173 min
  Start Time:   08:00:00
  End Time:     10:53:00
  Solver:       ortools_tsp
  Feasible:     T

In [37]:
# Show tradeoffs
print("\n" + "=" * 70)
print("TRADEOFF ANALYSIS")
print("=" * 70)

# Highest priority dispatch
highest_priority = max(ranked_dispatches, key=lambda d: d.total_priority)
print(f"\n📈 Highest Priority: {highest_priority.strategy}")
print(f"   Priority: {highest_priority.total_priority:,.0f} | Distance: {highest_priority.total_distance_km:.1f} km")

# Shortest distance dispatch
shortest_dist = min(ranked_dispatches, key=lambda d: d.total_distance_km)
print(f"\n🛣️ Shortest Distance: {shortest_dist.strategy}")
print(f"   Priority: {shortest_dist.total_priority:,.0f} | Distance: {shortest_dist.total_distance_km:.1f} km")


TRADEOFF ANALYSIS

📈 Highest Priority: greedy_efficiency
   Priority: 2,000,084 | Distance: 85.0 km

🛣️ Shortest Distance: greedy_best_fit
   Priority: 64 | Distance: 24.6 km


In [38]:
# Export results to JSON
export_path = DISPATCH_DIR / "ranked_dispatches_with_routes.json"
export_dispatches_with_routes(ranked_dispatches, export_path)
print(f"\n✓ Exported dispatches with routes to:")
print(f"  {export_path}")

# Export summary to CSV
csv_path = DISPATCH_DIR / "dispatch_summary.csv"
results_df.to_csv(csv_path, index=False)
print(f"\n✓ Exported summary CSV to:")
print(f"  {csv_path}")


✓ Exported dispatches with routes to:
  c:\Users\Santi\Desktop\CV\portafolio\Eco-Bags-Delivery-Optimizer\output\dispatches\ranked_dispatches_with_routes.json

✓ Exported summary CSV to:
  c:\Users\Santi\Desktop\CV\portafolio\Eco-Bags-Delivery-Optimizer\output\dispatches\dispatch_summary.csv


In [39]:
# Show sample of exported JSON
print("\n📄 Sample Export Format:")
with open(export_path, "r") as f:
    data = json.load(f)
    if data["dispatches"]:
        sample = data["dispatches"][0]
        print(json.dumps({
            "candidate_id": sample["candidate_id"],
            "strategy": sample["strategy"],
            "order_count": sample["order_count"],
            "total_priority": sample["total_priority"],
            "total_distance_km": sample["total_distance_km"],
            "total_duration_minutes": sample["total_duration_minutes"],
            "route": {
                "total_distance_km": sample["route"]["total_distance_km"],
                "feasible": sample["route"]["feasible"],
                "solver_used": sample["route"]["solver_used"],
                "stops_count": len(sample["route"]["stops"]),
            }
        }, indent=2))


📄 Sample Export Format:
{
  "candidate_id": "DISP-20260120-GREEDY-B27D",
  "strategy": "greedy_zone_caba",
  "order_count": 3,
  "total_priority": 1000140.51,
  "total_distance_km": 50.93,
  "total_duration_minutes": 149,
  "route": {
    "total_distance_km": 50.93,
    "feasible": true,
    "solver_used": "ortools_tsp",
    "stops_count": 4
  }
}


## Summary

This notebook demonstrated:

1. **Setup**: Loaded dispatch candidates from Phase 4 and routing configuration
2. **Single Dispatch Demo**: Walked through route optimization for one candidate
3. **Distance Matrix**: Visualized pairwise distances using Haversine formula
4. **Nearest Neighbor**: Baseline heuristic for comparison
5. **OR-Tools TSP**: Optimized solution using metaheuristics
6. **VRPTW**: Demonstrated time window handling
7. **Batch Optimization**: Optimized routes for all dispatch candidates
8. **Results Table**: Interactive summary with ranking
9. **Visualizations**: Charts comparing distance, duration, efficiency
10. **Route Maps**: Folium maps for top dispatches

### Key Outputs

| Output | Location |
|--------|----------|
| Ranked dispatches JSON | `output/dispatches/ranked_dispatches_with_routes.json` |
| Summary CSV | `output/dispatches/dispatch_summary.csv` |
| Top dispatch map | `output/maps/top_dispatch_route.html` |
| Single-zone route map | `output/maps/best_single_zone_route.html` |
| Comparison map | `output/maps/top3_routes_comparison.html` |

### Next Steps

The optimized routes are ready for:
- **Dispatch confirmation**: Select the best candidate for execution
- **Driver briefing**: Generate turn-by-turn directions
- **Time window validation**: Ensure all constraints are met
- **Performance tracking**: Compare actual vs estimated metrics